In [1]:
import wandb
from lstm import *
from torch.optim.lr_scheduler import StepLR

In [2]:
BS = 256
LR = 1e-8
EPOCHS = 100

In [3]:
# load glove embeddings
glove_obj = GloVe(name='840B', dim=300)

tokenizer = LSTMTokenizer(glove_obj=glove_obj, special_tokens={'pad': '<PAD>', 'eos': '<EOS>', 'bos': '<BOS>'})

# load model
model = LSTMModel(glove_obj=glove_obj)

# initialize data preparer for seq2seq training
data_prep = DataPreparer(tokenizer, 'knkarthick/dialogsum', 'next_word_prediction', is_lowercase=True)

# get train and test dataloaders
train_dataloader = data_prep('train', 'dialogue', batch_size=BS, shuffle=True)
test_dataloader = data_prep('validation', 'dialogue', batch_size=BS, shuffle=True)


Using custom data configuration knkarthick--dialogsum-b0174fca0a26ed84
Found cached dataset csv (/Users/bugrahamzagundog/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-b0174fca0a26ed84/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration knkarthick--dialogsum-b0174fca0a26ed84
Found cached dataset csv (/Users/bugrahamzagundog/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-b0174fca0a26ed84/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
device = 'cpu'

scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

def train(model, train_loader):
    wandb.init('lstm_tuner', project='chat2learn', config={'batch_size': BS, 
                                                           'learning_rate': LR, 
                                                           'epochs': EPOCHS})
    # move model to device
    model.to(device)

    model.train()
    for epoch in range(EPOCHS):
        for idx, (x, y) in enumerate(train_loader, start=1):
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y[:, 0])
            loss.backward()
            optimizer.step()
            scheduler.step()

            #print(f'Epoch: {epoch + 1}/{EPOCHS}, Batch: {idx}/{len(train_loader)}, Loss: {loss.item():.4f}')

            wandb.log({'epochs': epoch,
                       'learning_rate': scheduler.get_last_lr(),
                       'loss': loss.item()})

train(model, train_dataloader)

epochs,▁▁▁▁▁▁▁▁▁▁▁
loss,▁▇▅█▇▄▆▆▇▄▄
epochs,0
loss,14.60088


KeyboardInterrupt: 

wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.
